# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.




COMMIT AFTER EACH INSERT!!!!!!!

 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [2]:
## Connect to DB server on AWS
import mysql.connector
import config

cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
)
cursor = cnx.cursor()

In [3]:
db_name = 'yelp_reviews'

In [ ]:
cursor.execute("CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(db_name))
cursor.close()
cnx.close()

In [4]:
cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
        database = db_name
)
cursor = cnx.cursor()

In [ ]:
# Create a table for the Businesses
TABLES = {};
TABLES["hotels"] = ("""
      CREATE TABLE hotels (
      business_id varchar(50) NOT NULL UNIQUE,
      name varchar(100) NOT NULL,
      review_count int(4) NOT NULL,
      rating float(4) NOT NULL,
      price varchar(5),
      PRIMARY KEY (business_id)
      )
""")

In [ ]:
# Create a table for the reviews
TABLES["reviews"] = ("""
      CREATE TABLE reviews (
      review_id varchar(50) NOT NULL UNIQUE,
      business_id varchar(50) NOT NULL,
      review_text varchar(500) NOT NULL,
      PRIMARY KEY (review_id),
      FOREIGN KEY (business_id) REFERENCES hotels(business_id)
      )
""")

In [ ]:
# def create_table(TABLES):
for table_name in TABLES:
    table_description = TABLES[table_name]
    cursor.execute(table_description)

In [5]:
cursor.close()
cnx.close()

## Part 2: Create ETL pipeline for the business data from the API

In [31]:
import requests

client_id = 'UPf5tdRgr65dzjOVrAbSnQ'
api_key = 'G_ZNKs8WP13zcjHp0oHpw1760A0NB-IPNhlX27GtQp0tGgE9sDLhh-CFmcAuxtwHgBIgfTp4V0jNHWwWukAzZuWKFv05HocGeRvBxQKNPEgo-Oqkn38-9c1yUFOzXXYx'

In [ ]:
# Write a function to make a call to the yelp API
term = 'Hotels'
location = 'Manhattan NY'    
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': 50
             }

def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    yelp_data = response.json()
    return yelp_data

In [ ]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB

def parse_results(yelp_data):
    for i in yelp_data['businesses']:
        if len(i.keys()) < 16:
            i['price'] = ''

    hotel_list = []
    for hotel in yelp_data['businesses']:
        if hotel['price']:
            hotel_data = {}
            hotel_data['business_id'] = hotel['id']
            hotel_data['name'] = hotel['name']
            hotel_data['review_count'] = hotel['review_count']
            hotel_data['rating'] = hotel['rating']
            hotel_data['price'] = hotel['price']
            hotel_list.append(hotel_data)
        else:
            continue
    return hotel_list

In [ ]:
# Write a function to take your parsed data and insert it into the DB
def hotels_insert(hotel_list):
    cnx = mysql.connector.connect(
            host = config.host,
            user = config.user,
            password = config.password,
            database = db_name
    )
    cursor = cnx.cursor()
    hotels = []
    for hotel in hotel_list:
        hotels_tuple =(hotel['business_id'], hotel['name'], hotel['review_count'], hotel['rating'], hotel['price'])
        hotels.append(hotels_tuple)

    insert_query = """
    INSERT INTO hotels (business_id, name, review_count, rating, price) VALUES (%s, %s, %s, %s, %s)
    """

    cursor.executemany(insert_query, hotels)
    cnx.commit()
    cnx.close()

In [ ]:
# Write a script that combines the three functions above into a single process.
import time



def all_results(url_params, api_key):
    num = yelp_data['total']
    print('{} total matches found.'.format(num))
    cur = 0

    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
        hotels_insert(parsed_results)
        time.sleep(1)
        cur += 50
        
all_results(url_params, api_key)

## Part 3: Create ETL pipeline for the restaurant review data from the API

In [7]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
        database = db_name
)
cursor = cnx.cursor()
cursor.execute("""
SELECT
    name,
    business_id
FROM
    yelp_reviews.hotels
;
""")

list_bizIDs = []
for hotel in cursor.fetchall():
    list_bizIDs.append(hotel[1])

In [8]:
list_bizIDs

['--Uw7Ex3m2011N1hjXMsaA',
 '-2MFbrbn33gM_9nX9gvkkA',
 '-3zJg8bweB4UQL1_kQ1IgQ',
 '-6KiL27UCLZkSBUdc4JlgQ',
 '-aIIpbQx0PW8W4yHV2_Wcg',
 '-C9Tu-dcjZegsoO0sE45hQ',
 '-CrE2fB6jY2nkRWv-zmE3g',
 '-E4gSxVCHZ9hT7vAPsBP8A',
 '-g3Dsx0pJk6z9MwBKdjrtg',
 '-h11FHJlDo9gJXqciVPrRQ',
 '-Hneo47XIw4irjQBdPMecA',
 '-kcnNSl3A2rU1Fo8DQWGgA',
 '-M5r6zHWpxTuCb7dkfj9Hg',
 '-QHkY8MZRtIBbQMX_CZlnw',
 '-Wv7dOngCOJGASogrSTYMw',
 '006_rHOAhT87Xf6XoY_IxQ',
 '01p7HPenHY29MRSmzSh4MA',
 '02vF5JJNEsx0Z0Ala3EGHw',
 '03EFU3OR6j085vqGAXwVbA',
 '05aE1rxaCuPm8Gyn93nwFA',
 '063UhfsNjEcCzoQRpjIjHQ',
 '0fCtmvmDVhFBj3LyqVWQDg',
 '0gUKWcaa412zc5LFCbE8kA',
 '0iIaLaMfbWjZ0NktXu3fRQ',
 '0KyT5BS9ckMzkhnlnlfmQA',
 '0KZPOFvklUWvIJQIcJ4KeA',
 '0n7zwam7LYCXY41xz34M8g',
 '0PasC1jGgrK5zfpxsOS-UQ',
 '0RZpWHLjlR8Wll3MoED95w',
 '0SrF_WcRetbGWlps7ZhJNw',
 '0tYMltweY813Iicd8NKATA',
 '0UBRdmNVs9NHkasIFbEQDA',
 '0xp3lHJWVIH-dSxg-uMCUg',
 '0ZevFzq9pxl1r5ZIUVjlnw',
 '1-WJs7bMXXModhjcszw8ng',
 '11i9MWaquC2cX0W_edr6ow',
 '11sSXZAiqPA3UrNurkR8Hg',
 

In [32]:
# write a function that takes a business id 
# and makes a call to the API for reivews

# def yelp_call_reviews(api_key):
count = 0
reviews_list = []
for i_d in list_bizIDs:
    url = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(i_d)
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers)
    reviewz_data = response.json()['reviews']
    for rev in reviewz_data:
        rev['business_id'] = i_d
    reviews_list.append(reviewz_data)
    count += 1
    print(f"The {count} iteration was successful. Hotel ID: {i_d}")
#     return review_data

The 1 iteration was successful. Hotel ID: --Uw7Ex3m2011N1hjXMsaA
The 2 iteration was successful. Hotel ID: -2MFbrbn33gM_9nX9gvkkA
The 3 iteration was successful. Hotel ID: -3zJg8bweB4UQL1_kQ1IgQ
The 4 iteration was successful. Hotel ID: -6KiL27UCLZkSBUdc4JlgQ
The 5 iteration was successful. Hotel ID: -aIIpbQx0PW8W4yHV2_Wcg
The 6 iteration was successful. Hotel ID: -C9Tu-dcjZegsoO0sE45hQ
The 7 iteration was successful. Hotel ID: -CrE2fB6jY2nkRWv-zmE3g
The 8 iteration was successful. Hotel ID: -E4gSxVCHZ9hT7vAPsBP8A
The 9 iteration was successful. Hotel ID: -g3Dsx0pJk6z9MwBKdjrtg
The 10 iteration was successful. Hotel ID: -h11FHJlDo9gJXqciVPrRQ
The 11 iteration was successful. Hotel ID: -Hneo47XIw4irjQBdPMecA
The 12 iteration was successful. Hotel ID: -kcnNSl3A2rU1Fo8DQWGgA
The 13 iteration was successful. Hotel ID: -M5r6zHWpxTuCb7dkfj9Hg
The 14 iteration was successful. Hotel ID: -QHkY8MZRtIBbQMX_CZlnw
The 15 iteration was successful. Hotel ID: -Wv7dOngCOJGASogrSTYMw
The 16 iteration wa

The 125 iteration was successful. Hotel ID: 9TK6ea3GR2B6H6jErPRlTw
The 126 iteration was successful. Hotel ID: 9Yf0Oe9uCx8YmOjYROvZQg
The 127 iteration was successful. Hotel ID: a-09aMPYUVMVxfzuvfTKTQ
The 128 iteration was successful. Hotel ID: a3qXia8QjzuoFFVCje20_g
The 129 iteration was successful. Hotel ID: AbDtxVq8o8Z-NEB57Y0LjA
The 130 iteration was successful. Hotel ID: aCruJ1jAhr6iluwWLqP_vw
The 131 iteration was successful. Hotel ID: AD3KTAdvOyUYAdMBErnEuQ
The 132 iteration was successful. Hotel ID: adpehB-xkZzcmH4iYMMPNA
The 133 iteration was successful. Hotel ID: aFMDFPf7-YgeoNHDL6a-6A
The 134 iteration was successful. Hotel ID: aidY4r8pI2oNKQXbKfbHeQ
The 135 iteration was successful. Hotel ID: aNrCW0EKEMc9UIVgvXrhrg
The 136 iteration was successful. Hotel ID: APJKZ82_3GUfFirlUw8RGg
The 137 iteration was successful. Hotel ID: aQwe7venHcag8PKP2W0-rg
The 138 iteration was successful. Hotel ID: asGuo42LX_hfLBHpvAtJiQ
The 139 iteration was successful. Hotel ID: AUIyuc1nqTi1F2ZObs

The 248 iteration was successful. Hotel ID: FE6EtHq7obwHV_UQqIm4Zw
The 249 iteration was successful. Hotel ID: fEqSedl1uUUfpmfs3mMiwQ
The 250 iteration was successful. Hotel ID: FfdQVbXd9zyMcaOYAqmWZA
The 251 iteration was successful. Hotel ID: FfMA5gVGqCzrpD97EgmMeA
The 252 iteration was successful. Hotel ID: fgywbfvGfW3OdtVqjU4JaA
The 253 iteration was successful. Hotel ID: fiUIhWlDRSNxSjZw9WPHLw
The 254 iteration was successful. Hotel ID: Fj8l2e9aCOx6-aWSxBeG3A
The 255 iteration was successful. Hotel ID: FLgyL54mvP5-ZOGN5KV-_w
The 256 iteration was successful. Hotel ID: fMJOgRdSQ2ml1ngI1NWOxQ
The 257 iteration was successful. Hotel ID: fo5IyD5Vsz6g85a_fAexmQ
The 258 iteration was successful. Hotel ID: fT0XWIMIjZsKUL1VYdxTxA
The 259 iteration was successful. Hotel ID: fuUWO76l1oo-t7QFR1OLWA
The 260 iteration was successful. Hotel ID: fVnTJoGDJg3spEHtp9g91w
The 261 iteration was successful. Hotel ID: fVUuJd-ZR5DUhzjdI6mGRQ
The 262 iteration was successful. Hotel ID: FwHTiMjWED2CkDAZfq

The 371 iteration was successful. Hotel ID: Kgwal2tSiAB-ImzcV4Z24Q
The 372 iteration was successful. Hotel ID: kGX54tkc0zfT5ZquEPtwuQ
The 373 iteration was successful. Hotel ID: KhQKQha4hh5ob1q6CkH9UQ
The 374 iteration was successful. Hotel ID: KJs1wvcD0OeSlbEdYXioMg
The 375 iteration was successful. Hotel ID: KKhFd2VBfIj5n4t0rYVMYQ
The 376 iteration was successful. Hotel ID: KLcc7Yc-MI1p_mJKixOXGA
The 377 iteration was successful. Hotel ID: KO4PE3N_pKIFGbGAjqROvg
The 378 iteration was successful. Hotel ID: kP4k-GO4ETgstvNug3q31g
The 379 iteration was successful. Hotel ID: KqVuMQuOj5lAHU4D5Q-yHw
The 380 iteration was successful. Hotel ID: kRsNwqOj2Lj0OYxKRsUtsA
The 381 iteration was successful. Hotel ID: kRUF_Vt_zsvENf-xBAU2KA
The 382 iteration was successful. Hotel ID: ks299naLzw8CYdVlLnqPXQ
The 383 iteration was successful. Hotel ID: kt1unOwmWe9eLFSC2LXcuw
The 384 iteration was successful. Hotel ID: kUe6ZK09SMCpWKgapxbbNQ
The 385 iteration was successful. Hotel ID: kumbWukbwNktxWQB4_

The 494 iteration was successful. Hotel ID: p3VpauF_yu4TP3MFEbN9TA
The 495 iteration was successful. Hotel ID: pEELu1WWqMV6a3ygd_z57w
The 496 iteration was successful. Hotel ID: PgFl4uTnEO_37qZYZFpdEg
The 497 iteration was successful. Hotel ID: PgkXeq0fYLjn5hFGk15ekA
The 498 iteration was successful. Hotel ID: pi2sOXrCKVpKdPMaSwJ-_Q
The 499 iteration was successful. Hotel ID: PJ-4oK82pJtuqpJk7LVlXw
The 500 iteration was successful. Hotel ID: pJfy-huJiMxJVc-4fWDYuQ
The 501 iteration was successful. Hotel ID: PkJjSD067n6o9nnmyWNtQQ
The 502 iteration was successful. Hotel ID: PmmTXis1gCL34mg2bZ9gtw
The 503 iteration was successful. Hotel ID: PmVTYZy6_5OuT1gwhC9C6g
The 504 iteration was successful. Hotel ID: PUT2TmGaFIr37ey3m8_THA
The 505 iteration was successful. Hotel ID: pW91FEkwCIW6KKcV7nFeOQ
The 506 iteration was successful. Hotel ID: pY-DMHCPIRifZo9Cay1Low
The 507 iteration was successful. Hotel ID: pYVmFx5H42agzkaULfGaUw
The 508 iteration was successful. Hotel ID: q-6jXyKUYc0pQmvyQo

The 617 iteration was successful. Hotel ID: ue1H_7beop4Ks2PmkjU_Mg
The 618 iteration was successful. Hotel ID: uK3Wug7R0YylT_u0QWZjUg
The 619 iteration was successful. Hotel ID: Uk4tD6CZCBXmOthhPiIVvA
The 620 iteration was successful. Hotel ID: uLFLKwkpIdydKNVHB9Rdhw
The 621 iteration was successful. Hotel ID: UNEvhBHudaU7OTgbR3q5Ug
The 622 iteration was successful. Hotel ID: uNP7_-y7IeK53uIjRYu_iQ
The 623 iteration was successful. Hotel ID: Uoioa4Sv9AZ9HAt5jeV2Gg
The 624 iteration was successful. Hotel ID: UQD85Z7Hiq_1Q3Jl7Jx-HQ
The 625 iteration was successful. Hotel ID: uQnrxRcDpu65pY2clPpLMQ
The 626 iteration was successful. Hotel ID: UReqUKgLehSHPkuWfFMoig
The 627 iteration was successful. Hotel ID: UuoI_umL3OlADKiLDVKpzQ
The 628 iteration was successful. Hotel ID: UYMVIyEwjty3OelDsjy4GA
The 629 iteration was successful. Hotel ID: UyneWnO7UESy04cq6GbZcA
The 630 iteration was successful. Hotel ID: uy_HQ7wDlkNDq_y8trbaQA
The 631 iteration was successful. Hotel ID: UZfIZOlv5wMagsCx-S

The 740 iteration was successful. Hotel ID: z0Otqcc5fmo0tMA86Z2quw
The 741 iteration was successful. Hotel ID: Z1aUHQ0qdwCdyA9UvbpMRg
The 742 iteration was successful. Hotel ID: Z3UYN02VD8zBLGQwGBphtw
The 743 iteration was successful. Hotel ID: Z4t_Wsmjgi8I5n1xPSQWTA
The 744 iteration was successful. Hotel ID: z5LyFhHkXNefCZ7gvDepZQ
The 745 iteration was successful. Hotel ID: Z9L628nPMRC6mmWxas95dA
The 746 iteration was successful. Hotel ID: zafgIOFJ8VAdx_lxmPQQEw
The 747 iteration was successful. Hotel ID: ZDJMWEI7GmMTaaZwenWt5Q
The 748 iteration was successful. Hotel ID: ZDp1ysOVW3xQJc4SuQGSCg
The 749 iteration was successful. Hotel ID: Zg9KkQP0Xq1QGmIKTL2pyw
The 750 iteration was successful. Hotel ID: ZHc2G1IiT3Ex3yS3X_qQyw
The 751 iteration was successful. Hotel ID: Zhe3lXdqS3lxgCjqRNViLw
The 752 iteration was successful. Hotel ID: ZK8zDqB_Us_QyYrPdfGDNA
The 753 iteration was successful. Hotel ID: ZKXrTleagEza0o4XlTyZeg
The 754 iteration was successful. Hotel ID: Zmz06iDh5ywLsLoCAt

In [34]:
reviewz_data[-1]

{'id': 'SBMmCYTJfQSV4xL8ubnjVw',
 'url': 'https://www.yelp.com/biz/the-gotham-hotel-new-york?adjust_creative=UPf5tdRgr65dzjOVrAbSnQ&hrid=SBMmCYTJfQSV4xL8ubnjVw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=UPf5tdRgr65dzjOVrAbSnQ',
 'text': 'Pros: the hotel has a great location in walking distance to Times Square, Lincoln Center, Rockefeller Plaza and Grand Central. The staff was friendly but...',
 'rating': 3,
 'time_created': '2019-04-22 09:07:42',
 'user': {'id': 'xePXJUbxy0K0SikICggC5g',
  'profile_url': 'https://www.yelp.com/user_details?userid=xePXJUbxy0K0SikICggC5g',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/photo/lVlRGSp-YdD8HlU_PR1OwQ/o.jpg',
  'name': 'Mary F.'},
 'business_id': '_vteB78WnOiuOOQT_38X8w'}

In [36]:
# Write a function to parse out the relevant information from the reviews

parsed_reviews = []
for reviews in reviews_list:
    for review in reviews:
        review_data = {}
        review_data['review_id'] = review['id'] 
        review_data['review_text'] = review['text']
        review_data['business_id'] = review['business_id']
        parsed_reviews.append(review_data)
print(parsed_reviews[-1])
print(len(parsed_reviews))

{'review_id': 'SBMmCYTJfQSV4xL8ubnjVw', 'review_text': 'Pros: the hotel has a great location in walking distance to Times Square, Lincoln Center, Rockefeller Plaza and Grand Central. The staff was friendly but...', 'business_id': '_vteB78WnOiuOOQT_38X8w'}
2272


In [37]:
# write a function to insert the parsed data into the reviews table
# def reviews_insert(parsed_reviews):
cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
        database = db_name
)
cursor = cnx.cursor()
reviews = []
for review in parsed_reviews:
    reviews_tuple =(review['review_id'], review['review_text'], review['business_id'])
    reviews.append(reviews_tuple)

insert_query = """
INSERT INTO reviews (review_id, review_text, business_id) VALUES (%s, %s, %s)
"""

cursor.executemany(insert_query, reviews)
cnx.commit()
cnx.close()

In [ ]:
# combine the functions above into a single script  

## Part 4: Write SQL queries that will answer the questions posed. 

In [38]:
# create connection
cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
        database = db_name
)
cursor = cnx.cursor()

In [ ]:
# execute SQL queries

In [44]:
# Which are the 5 most reviewed businesses?
cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
        database = db_name
)
cursor = cnx.cursor()

cursor.execute("""
SELECT
    name,
    review_count
FROM
    yelp_reviews.hotels
ORDER BY
    review_count DESC
LIMIT
    5
;
""")
cursor.fetchall()

[('Hotel Pennsylvania', 1418),
 ('Hotel Chantelle', 1296),
 ('Hudson Hotel - Central Park', 1251),
 ('YOTEL New York', 1194),
 ('New York Marriott Marquis', 954)]

In [48]:
# What is the highest rating recieved in your data set and how many businesses have that rating?

cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
        database = db_name
)
cursor = cnx.cursor()
cursor.execute("""
SELECT
    name,
    rating,
    review_count
FROM
    yelp_reviews.hotels
WHERE
    rating = (
        SELECT
            MAX(rating)
        FROM
            yelp_reviews.hotels
    )
;
""")
cursor.fetchall()

[('Hotel Mimosa', 5.0, 20),
 ('The Harlem Flop House', 5.0, 13),
 ('The Lafayette House', 5.0, 7),
 ('Adventure Hostels JFK', 5.0, 1),
 ('AVE Clifton', 5.0, 4),
 ('W Hotels - The Store', 5.0, 1)]

In [52]:
# Return the text of the reviews for the most reviewed restaurant.

cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.password,
        database = db_name
)
cursor = cnx.cursor()
cursor.execute("""
SELECT
    review_text
FROM
    yelp_reviews.hotels h
JOIN
    yelp_reviews.reviews r
ON
    h.business_id = r.business_id
WHERE
    review_count = (
        SELECT
            MAX(review_count)
        FROM
            yelp_reviews.hotels
    )
;
""")
cursor.fetchall()

[('I stayed here during the US Open in 2018.\n\nI got to the front desk after a red-eye coast-to-coast flight and a hired car from Newark at my stated check-in...',),
 ('Have you ever woken up from a horrible nightmare, taken a deep breath, opened your eyes, and realized to your great dismay that, "the nightmare is reality....',),
 ("I always start with the good. This hotel just went through a significant renovation to their rooms and they did a great job. It's an old hotel, but I love...",)]

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [ ]:
def parse_results(results):
    
    return parsed_results

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time



def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
        db_insert(parsed)
        time.sleep() #Wait a second
        cur += 50

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```

### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)

In [ ]:
# Which are the 5 most reviewed businesses?
cursor.execute()